<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/505_EPOv2_decision_node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 1. ✅ What This Node Already Nails

### 🔹 Correct separation of concerns

This node **does not re-analyze data** and **does not compute stats**. It strictly:

* Consumes analysis
* Applies policy
* Emits decisions

That’s exactly right.

You avoided the common mistake of letting “decision logic creep into analysis logic.”

---

### 🔹 Proper merge of calculated + loaded analyses

This is *very good*:

```python
analysis_lookup = state.get("analysis_lookup", {})
calculated_analyses = state.get("calculated_analyses", [])

for analysis in calculated_analyses:
    exp_id = analysis.get("experiment_id")
    if exp_id:
        analysis_lookup[exp_id] = analysis
```

This ensures:

* Fresh calculations override stale stored analysis
* Downstream nodes see a unified view

This is **state-centric orchestration done right**.

---

### 🔹 Symmetric handling of single vs portfolio scope

Both branches behave consistently:

| Scope     | Behavior                              |
| --------- | ------------------------------------- |
| Single    | Evaluate exactly one experiment       |
| Portfolio | Evaluate only those needing decisions |
| Fallback  | Safe default for partial pipelines    |

This symmetry makes the system predictable.

---

## 2. 🔍 Subtle but Important State Issue

### ❗ You are mutating `analysis_lookup` in-place

This line is the culprit:

```python
analysis_lookup = state.get("analysis_lookup", {})
```

If `analysis_lookup` is a **reference** to the state object (which it usually is), you are mutating shared state without explicitly returning it.

Right now this *works* because:

* You don’t rely on the original `analysis_lookup` later
* Python dicts are mutable

But this can cause **silent coupling** between nodes later.

---

### ✅ Recommended (safe) pattern

Make the merge explicit and return it:

```python
analysis_lookup = dict(state.get("analysis_lookup", {}))

for analysis in calculated_analyses:
    exp_id = analysis.get("experiment_id")
    if exp_id:
        analysis_lookup[exp_id] = analysis
```

And then **return it**:

```python
return {
    "analysis_lookup": analysis_lookup,
    "generated_decisions": generated_decisions,
    "errors": errors
}
```

This keeps your orchestrator **pure, explicit, and testable**.

---

## 3. ⚠️ Two Small Fixes You Should Make

These are easy and worth doing now.

---

### ❗ Fix 1: Require `analysis_lookup` explicitly

Right now you check:

```python
if not definitions_lookup:
    return error
```

But you don’t check for `analysis_lookup`.

In single-experiment mode, this can silently do nothing if:

* Definitions exist
* Analysis is missing

#### ✅ Add this guard:

```python
if not analysis_lookup and not calculated_analyses:
    return {
        "errors": errors + ["decision_evaluation_node: analysis required. Run statistical_analysis_node first."]
    }
```

This prevents “empty success.”

---

### ❗ Fix 2: Align error messages with other nodes

Most of your nodes follow this pattern:

```python
"<node_name>: <clear actionable message>"
```

This one already mostly does — just make it consistent everywhere.

Example tweak:

```python
"decision_evaluation_node: definitions_lookup required. Run data_loading_node first."
```

is perfect.
Apply the same clarity for missing analysis.

---

## 4. 🧠 Why This Node Completes the System

This node is where your architecture **crosses the line** from:

> “A data analysis pipeline”

to

> **“An organizational decision system”**

Because now you have:

* Evidence → Analysis
* Analysis → Policy
* Policy → Action
* Action → Review date
* Review date → Reversal triggers

That’s a *full governance loop*.

---

## Final Verdict

### This node is:

* ✅ Correctly scoped
* ✅ State-aware
* ✅ Policy-driven
* ✅ Deterministic
* ✅ CEO-defensible

And with **two small safety tweaks**, it becomes production-grade.




In [ ]:

def decision_evaluation_node(
    state: ExperimentationPortfolioOrchestratorState,
    config: Optional[ExperimentationPortfolioOrchestratorConfig] = None
) -> Dict[str, Any]:
    """
    Decision Evaluation Node: Evaluate experiments and generate decision recommendations.

    For portfolio-wide: Evaluates experiments needing decisions.
    For single experiment: Evaluates the experiment if analysis exists.

    Args:
        state: Current state
        config: Optional config (uses decision thresholds from config)
    """
    errors = state.get("errors", [])
    goal = state.get("goal", {})
    scope = goal.get("scope", "portfolio_wide")

    # Use provided config or create default
    if config is None:
        config = ExperimentationPortfolioOrchestratorConfig()

    # Get required data from state
    definitions_lookup = state.get("definitions_lookup", {})
    portfolio_lookup = state.get("portfolio_lookup", {})
    decisions_lookup = state.get("decisions_lookup", {})

    # Get analysis (from loaded or calculated)
    analysis_lookup = state.get("analysis_lookup", {})
    calculated_analyses = state.get("calculated_analyses", [])

    # Merge calculated analyses into lookup
    for analysis in calculated_analyses:
        exp_id = analysis.get("experiment_id")
        if exp_id:
            analysis_lookup[exp_id] = analysis

    if not definitions_lookup:
        return {
            "errors": errors + ["decision_evaluation_node: definitions_lookup required. Run data_loading_node first."]
        }

    try:
        generated_decisions = []

        if scope == "single_experiment":
            # Single experiment evaluation
            experiment_id = state.get("experiment_id")
            if not experiment_id:
                return {
                    "errors": errors + ["decision_evaluation_node: experiment_id required for single experiment evaluation"]
                }

            definition = definitions_lookup.get(experiment_id)
            analysis = analysis_lookup.get(experiment_id)
            portfolio_entry = portfolio_lookup.get(experiment_id) if portfolio_lookup else None

            if definition and analysis:
                # Check if decision already exists
                if experiment_id not in decisions_lookup:
                    decision = evaluate_experiment_decision(
                        experiment_id=experiment_id,
                        definition=definition,
                        analysis=analysis,
                        portfolio_entry=portfolio_entry,
                        config=config
                    )
                    generated_decisions.append(decision)
        else:
            # Portfolio-wide evaluation
            analyzed_experiments = state.get("analyzed_experiments", [])

            if analyzed_experiments:
                generated_decisions = evaluate_experiments_needing_decisions(
                    analyzed_experiments=analyzed_experiments,
                    definitions_lookup=definitions_lookup,
                    analysis_lookup=analysis_lookup,
                    portfolio_lookup=portfolio_lookup,
                    decisions_lookup=decisions_lookup,
                    config=config
                )
            else:
                # Fallback: evaluate all experiments with analysis but no decision
                for exp_id, definition in definitions_lookup.items():
                    if exp_id in decisions_lookup:
                        continue  # Skip if decision exists

                    analysis = analysis_lookup.get(exp_id)
                    if analysis:
                        portfolio_entry = portfolio_lookup.get(exp_id) if portfolio_lookup else None
                        decision = evaluate_experiment_decision(
                            experiment_id=exp_id,
                            definition=definition,
                            analysis=analysis,
                            portfolio_entry=portfolio_entry,
                            config=config
                        )
                        generated_decisions.append(decision)

        return {
            "generated_decisions": generated_decisions,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"decision_evaluation_node: Unexpected error - {str(e)}"]
        }
